In [ ]:
import numpy as np
from scipy import stats

def one_sample_z_test(sample, pop_mean, pop_std=None, sample_std=None, alpha=0.05, alternative='two-sided'):
    """
    Perform a one-sample Z-test to determine if the sample mean differs from the population mean.
    
    Parameters:
    - sample: Array-like, the sample data
    - pop_mean: float, known population mean
    - pop_std: float, known population standard deviation (if None, uses sample std)
    - sample_std: float, sample standard deviation (only needed if pop_std is None)
    - alpha: float, significance level (default 0.05)
    - alternative: str, 'two-sided', 'less', or 'greater' (default 'two-sided')
    
    Returns:
    - Dictionary containing:
        'z_score': calculated z-score
        'p_value': calculated p-value
        'reject_null': boolean indicating whether to reject null hypothesis
        'ci': confidence interval for the mean difference
    """
    sample = np.asarray(sample)
    sample_mean = np.mean(sample)
    n = len(sample)
    if pop_std is not None:
        std_error = pop_std / np.sqrt(n)
    else:
        if sample_std is None:
            sample_std = np.std(sample, ddof=1) 
        std_error = sample_std / np.sqrt(n)
    z_score = (sample_mean - pop_mean) / std_error
    if alternative == 'two-sided':
        p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))
    elif alternative == 'less':
        p_value = stats.norm.cdf(z_score)
    elif alternative == 'greater':
        p_value = 1 - stats.norm.cdf(z_score)
    else:
        raise ValueError("alternative must be 'two-sided', 'less', or 'greater'")
    reject_null = p_value < alpha
    if alternative == 'two-sided':
        margin = stats.norm.ppf(1 - alpha/2) * std_error
        ci = (sample_mean - margin, sample_mean + margin)
    elif alternative == 'less':
        margin = stats.norm.ppf(1 - alpha) * std_error
        ci = (-np.inf, sample_mean + margin)
    else: 
        margin = stats.norm.ppf(1 - alpha) * std_error
        ci = (sample_mean - margin, np.inf)
    return {
        'z_score': z_score,
        'p_value': p_value,
        'reject_null': reject_null,
        'ci': ci,
        'sample_mean': sample_mean,
        'pop_mean': pop_mean,
        'n': n,
        'std_error': std_error,
        'alpha': alpha,
        'alternative': alternative
    }
if __name__ == "__main__":
    np.random.seed(42)
    sample_scores = np.random.normal(loc=77, scale=10, size=30)
    population_mean = 75
    population_std = 10 
    result = one_sample_z_test(sample_scores, pop_mean=population_mean, pop_std=population_std)
    print("One-Sample Z-Test Results")
    print("-------------------------")
    print(f"Sample size: {result['n']}")
    print(f"Sample mean: {result['sample_mean']:.2f}")
    print(f"Population mean: {result['pop_mean']}")
    print(f"Z-score: {result['z_score']:.4f}")
    print(f"P-value: {result['p_value']:.4f}")
    print(f"95% Confidence Interval: ({result['ci'][0]:.2f}, {result['ci'][1]:.2f})")
    print(f"Significance level (alpha): {result['alpha']}")
    print(f"Alternative hypothesis: {result['alternative']}")
    print(f"Reject null hypothesis? {'Yes' if result['reject_null'] else 'No'}")

One-Sample Z-Test Results
-------------------------
Sample size: 30
Sample mean: 75.12
Population mean: 75
Z-score: 0.0649
P-value: 0.9482
95% Confidence Interval: (71.54, 78.70)
Significance level (alpha): 0.05
Alternative hypothesis: two-sided
Reject null hypothesis? No
